In [ ]:
import serial, re, csv, time, uuid
from datetime import datetime
from pathlib import Path

#----------------------------------[old code]--------------------------------------------#

#config with controller
SERIAL_PORT = 'COM8'
BAUD_RATE = 9600
OUTPUT_DIR = Path("labeled_data")
OUTPUT_DIR.mkdir(exist_ok=True)
DURATION_SEC = 4        #record length per trial (seconds)
INTER_TRIAL_DELAY = 1.0 #seconds between trials

#regex patterns for gyro and acc lines
gyro_pattern = re.compile(r'Gyro\. X = (-?\d+), Y = (-?\d+), Z = (-?\d+)')
acc_pattern  = re.compile(r'Acc\. X = (-?\d+), Y = (-?\d+), Z = (-?\d+)')

#access serial connection
ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)

#-----------------------------------[new code]--------------------------------------------#

# The whole idea of this section is to optimize our data collection for loading the gestures. 
# Running this should allow us to record as many samples as we need for each gesture, and label
# the data at the same time. With the serialToCSV file from before, this would have taken ages.

# The output should be a single CSV file, with timestamps, labels, and position data for each sample.
# This should make building my algos a lot simpler.

def record_trial(session_id, trial_id, label, out_file):
    print(f"Get ready for trial {trial_id} label='{label}' ... starting in 2s")
    time.sleep(2)
    start_t = time.time()
    sample_idx = 0
    with open(out_file, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        while time.time() - start_t < DURATION_SEC:
            line = ser.readline().decode('utf-8', errors='ignore').strip()
            timestamp_iso = datetime.now().isoformat()
            timestamp_ms = int(time.time()*1000)
            gyro_match = gyro_pattern.match(line)
            if gyro_match:
                gyro = list(map(int, gyro_match.groups()))
                #read next line for acc
                acc_line = ser.readline().decode('utf-8', errors='ignore').strip()
                acc_match = acc_pattern.match(acc_line)
                if acc_match:
                    acc = list(map(int, acc_match.groups()))
                    row = [
                        session_id, trial_id, timestamp_iso, timestamp_ms,
                        sample_idx,
                        gyro[0], gyro[1], gyro[2],
                        acc[0], acc[1], acc[2],
                        label, ""
                    ]
                    writer.writerow(row)
                    sample_idx += 1
    print(f"Finished trial {trial_id}.")
    time.sleep(INTER_TRIAL_DELAY)

def main():
    session_id = datetime.now().strftime("%Y%m%d_%H%M%S") + "_" + str(uuid.uuid4())[:8]
    out_file = OUTPUT_DIR / f"session_{session_id}.csv"
    header = ['session_id','trial_id','timestamp_iso','timestamp_ms','sample_index',
              'gyro_x','gyro_y','gyro_z','acc_x','acc_y','acc_z','label','note']
    with open(out_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)

    print("Interactive labeled data collection.")
    print("Type labels you want to record. Type 'quit' to exit.")

    trial_counter = 0
    try:
        while True:
            label = input("Enter label (or 'quit'): ").strip()
            if label.lower() in ('quit'):
                break
            trial_counter += 1
            trial_id = f"t{trial_counter:03d}"
            record_trial(session_id, trial_id, label, out_file)
    except KeyboardInterrupt:
        print("\nInterrupted by user.")
    finally:
        ser.close()
        print(f"Saved: {out_file}")

if __name__ == "__main__":
    main()
